# Data Loading and Cleaning 

In [24]:
#importing necessary libraries

import numpy as np
import pandas as pd

# plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
#loading data 
slice_data = pd.read_csv('Slice Data.csv')

In [26]:
slice_data.shape

(1825, 9)

From the shape of the data frame **slice_data** we can see that the dataset has 1825 rows and 8 columns. 

In [27]:
slice_data.head() # looking at the first five rows

,Date,Flavor,Sold (Qty),Produced (Qty),Temperature (Avg.),Rain?,Waste (%),Waste (Qty),Revenue/ Profit
0,1/1/2023,Pepperoni,120,124,77.5,0,0.03,4,$720
1,1/1/2023,Cheese,105,106,77.5,0,0.01,1,$630
2,1/1/2023,Vegetariana,120,126,77.5,0,0.05,6,$840
3,1/1/2023,Primavera,60,61,77.5,0,0.01,1,$420
4,1/1/2023,Napolitana,45,46,77.5,0,0.02,1,$315


Looking at the first five rows it looks like we have sensible data based on the names of the columns so far. The next step we need to take involved looking into the data types of each column. 

In [28]:
slice_data.info() # looking at the data types of each column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                1825 non-null   object 
 1   Flavor              1825 non-null   object 
 2   Sold (Qty)          1825 non-null   int64  
 3   Produced (Qty)      1825 non-null   int64  
 4   Temperature (Avg.)  1825 non-null   float64
 5   Rain?               1825 non-null   int64  
 6   Waste (%)           1825 non-null   float64
 7   Waste (Qty)         1825 non-null   int64  
 8   Revenue/ Profit     1825 non-null   object 
dtypes: float64(2), int64(4), object(3)
memory usage: 128.4+ KB


By using the **.info()** method above, we are able to look at the data types of each column. 

**Takeaways**: 
- We have three data types within our dataset: Float64, Int64, and Object
- There seems to be no null values within our dataset. 

Now, since we are going to focus on a time series analysis we need to set our index to be the 'Date' column. This will allow us to analyze the behavior of our data how through time. 

In [29]:
slice_data["Date"] = pd.to_datetime(slice_data["Date"]) #converting to date time object 
slice_data = slice_data.set_index("Date") # setting the date as the index for time series analysis 

slice_data.head() #looking at our new dataframe with 'Date' column as the index

,Flavor,Sold (Qty),Produced (Qty),Temperature (Avg.),Rain?,Waste (%),Waste (Qty),Revenue/ Profit
Date,,,,,,,,
2023-01-01,Pepperoni,120,124,77.5,0,0.03,4,$720
2023-01-01,Cheese,105,106,77.5,0,0.01,1,$630
2023-01-01,Vegetariana,120,126,77.5,0,0.05,6,$840
2023-01-01,Primavera,60,61,77.5,0,0.01,1,$420
2023-01-01,Napolitana,45,46,77.5,0,0.02,1,$315


In [30]:
slice_data.info() # sanity check to make sure date is our new index

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1825 entries, 2023-01-01 to 2023-12-31
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Flavor              1825 non-null   object 
 1   Sold (Qty)          1825 non-null   int64  
 2   Produced (Qty)      1825 non-null   int64  
 3   Temperature (Avg.)  1825 non-null   float64
 4   Rain?               1825 non-null   int64  
 5   Waste (%)           1825 non-null   float64
 6   Waste (Qty)         1825 non-null   int64  
 7   Revenue/ Profit     1825 non-null   object 
dtypes: float64(2), int64(4), object(2)
memory usage: 128.3+ KB


Looking at the info method again, we see that we have a Datetime index which will help us analyze how our data behaves throughout time. Another thing that we are able to see is that our 'Revenue/ Proftit' column is of the object data type which doesnt make sense. Our next step is to convert the column to the appropriate data type. The reason why the column is being identified as an object is beacause of the '$' and ',' that is part of the values within the column causing it to be classified as a string object due to the combination of numbers and characters. To remove those characters we need to use the method replace that is part of the string object in pandas. 

In [31]:
# to convert Profit/Revenue column to a int64 data type 
slice_data['Revenue/ Profit'] = slice_data['Revenue/ Profit'].str.replace('$','',regex=True) #removing '$'
slice_data['Revenue/ Profit'] = slice_data['Revenue/ Profit'].str.replace(',','',regex=True) #removing ','
slice_data['Revenue/ Profit'] = slice_data['Revenue/ Profit'].astype('int64')

In [32]:
slice_data.info() #sanity check to make sure the data type of 'Revenue/ Profit' column is right 

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1825 entries, 2023-01-01 to 2023-12-31
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Flavor              1825 non-null   object 
 1   Sold (Qty)          1825 non-null   int64  
 2   Produced (Qty)      1825 non-null   int64  
 3   Temperature (Avg.)  1825 non-null   float64
 4   Rain?               1825 non-null   int64  
 5   Waste (%)           1825 non-null   float64
 6   Waste (Qty)         1825 non-null   int64  
 7   Revenue/ Profit     1825 non-null   int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 128.3+ KB


Now, we want to look at the time series aspect of our dataset. 

In [33]:
first_day = slice_data.index.min() 
last_day = slice_data.index.max()

first_day, last_day # pandas `Timestamp` objects

(Timestamp('2023-01-01 00:00:00'), Timestamp('2023-12-31 00:00:00'))

Using the **.min()** and **.max()** method on the index of the dataframe we are able to understand where our data begins and where it ends. 

Takeaways: 
- The first day within our data set is January 1st 2023. 
- The last day within our data set is December 31st 2023. 

In [34]:
last_day - first_day # range of data 

Timedelta('364 days 00:00:00')

From the Timedelta output we are able to see that we are working with data within the year 2023 which comes out to being 364 days. Now, our next step is looking for null values within our range of data. We will be doing so by suing the **.date_range()** method from pandas using *first_day, last_day, and frequency* set to 'D' (Days) in conjunction with the **.difference()** method to check for any missing values within a range.

In [35]:
full_range = pd.date_range(start=first_day, end=last_day, freq="D")

full_range.difference(slice_data.index) # checking for any null values within the date range

DatetimeIndex([], dtype='datetime64[ns]', freq=None)

Another method that can be used to check for null values within our data set is the **.isna()** method wich we will be using as a sanity check to confirm for any null values in our data set.

In [36]:
slice_data.isna().sum() # checking for any null values

Flavor                0
Sold (Qty)            0
Produced (Qty)        0
Temperature (Avg.)    0
Rain?                 0
Waste (%)             0
Waste (Qty)           0
Revenue/ Profit       0
dtype: int64

As shown above, we have no null values in our dataset. Our next step is to check for any duplicate values using the **.duplicate()** method. 

In [37]:
slice_data.duplicated().sum() #check for any duplicates

203

The number above shows us that we have 203 duplicated rows within our dataset. Before handling those duplicates we need to investigate the source of these duplicate values. We will be using the method **pd.set_option('display.max_rows', None)** to show all of the duplicated rows. 

In [38]:
pd.set_option('display.max_rows', None) # to show all rows
slice_data[slice_data.duplicated()] # looking at those duplicates

,Flavor,Sold (Qty),Produced (Qty),Temperature (Avg.),Rain?,Waste (%),Waste (Qty),Revenue/ Profit
Date,,,,,,,,
2023-01-29,Vegetariana,120,126,77.5,0,0.05,6,840
2023-02-02,Napolitana,45,46,77.0,0,0.02,1,315
2023-02-09,Napolitana,15,15,77.5,0,0.02,0,105
2023-02-14,Cheese,15,15,67.0,0,0.01,0,90
2023-02-19,Napolitana,15,15,77.5,0,0.02,0,105
2023-02-20,Pepperoni,60,61,78.5,0,0.02,1,360
2023-02-21,Pepperoni,45,46,78.0,0,0.03,1,270
2023-02-21,Cheese,15,15,78.0,0,0.01,0,90
2023-02-21,Napolitana,60,61,78.0,0,0.02,1,420


After looking into the 203 duplicated rows, I have decided to keep these duplicates as the duplicate values are asociated to different days. These duplicates, in turn, show the actual behavior of our data that can be useful in our time series analysis. 

In [41]:
#exporting cleaned dataset that will be used in EDA
slice_data.to_csv('slice_data_cleaned.csv', index=True)

Next Steps: **Exploratory Data Analysis (EDA)**